# 🤖 Janus + Azure OpenAI Assistant
Welcome! Ask any SQL-style question about your Janus Data Mesh (Trino-based).

### 🗂️ Connected Catalogs:
- `dh_captool_oem`
- `dh_pgcentral_pig`
- `backbone_alignops`
- `dh_pgrfx_fxa`
- `dh_pgcentral_futures`
- `dh_eqd3_prod`
- `risk_ops`
- `analytics_lake`
- `market_surveillance`
- `compliance_pivot`

### 💡 Example Queries You Can Try:
- *List all schemas in the `dh_captool_oem` catalog.*
- *What are the top 5 largest tables in the `analytics_lake` catalog?*
- *Show the row count of tables in `risk_ops` schema under `dh_eqd3_prod` catalog.*
- *Which users have access to the `market_surveillance` catalog?*
- *Give me column names for `trade_events` table in the `dh_pgcentral_futures` catalog.*
- *What tables are available under the `compliance_pivot` catalog?*

👉 Type `exit` to quit.

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chat_models import init_chat_model
from ai_tools import JanusQueryTool
from IPython.display import display, Markdown
import ipywidgets as widgets

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"

with open("ai_context.txt", "r") as f:
    ai_context = f.read()

print("Creating LLM...")
llm = init_chat_model(
    model="gpt-4o",
    model_provider="azure_openai"
)
print("LLM created")

janus_tool = JanusQueryTool()
tools = [
    Tool(
        name="JanusQueryTool",
        func=janus_tool.run,
        description="""
        Use this tool to run SQL-like queries on Janus data mesh.
        Use Trino SQL.
        The Janus data mesh contains enterprise data products.
        This is a multiple catalog setup.
        """
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    handle_parsing_errors=True
)

In [ ]:
query_box = widgets.Textarea(
    value='',
    placeholder='Type your full SQL-like data question here...',
    description='Query:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(
    description="Submit Query",
    button_style='success',
    tooltip='Click to submit query',
    icon='check'
)

output_box = widgets.Output()

def on_button_click(b):
    query = query_box.value.strip()
    if query.lower() == "exit":
        query_box.disabled = True
        submit_button.disabled = True
        with output_box:
            display(Markdown("✅ **Session ended. Thank you!**"))
        return
    if not query:
        return
    response = agent.run(
        f"Use this context to help you answer the query.\n\nContext:\n{ai_context}\n\nQuery:\n{query}"
    )
    with output_box:
        display(Markdown(f"**🗨️ Query:** `{query}`\n\n**💡 Response:**\n\n{response}"))
    query_box.value = ''

submit_button.on_click(on_button_click)

display(Markdown("### 📊 Janus + Azure OpenAI Assistant"))
display(query_box, submit_button, output_box)